In [1]:
import os
mingw_path = 'C:\\mingw-w64\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
import numpy as np
import pandas as pd
import xgboost
from sklearn import preprocessing
import time

In [2]:
feature_list = ['x','y','hour','weekday', 'day','month','year', 'accuracy']

In [3]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values
    df.y = df.y.values
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60)
    #for i in range(0,24):
    #    df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24)
    
    #df['w0'] = ((d_times.weekday + 0) % 7)
    #df['w1'] = ((d_times.weekday + 1) % 7)
    #df['w2'] = ((d_times.weekday + 2) % 7)
    #df['w3'] = ((d_times.weekday + 3) % 7)
    #df['w4'] = ((d_times.weekday + 4) % 7)
    #df['w5'] = ((d_times.weekday + 5) % 7)
    #df['w6'] = ((d_times.weekday + 6) % 7)
    df['weekday'] = d_times.weekday
    
    df['day'] = (d_times.dayofyear).astype(int)
    df['month'] = d_times.month
    df['year'] = (d_times.year - 2013)
    #df.accuracy = df.accuracy.values * fw[7]
    df['accuracy'] = np.log10(df.accuracy)
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30))
    #df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def mapkprecision(truthvalues, predictions):
    '''
    This is a faster implementation of MAP@k valid for numpy arrays.
    It is only valid when there is one single truth value. 

    m ~ number of observations
    k ~ MAP at k -- in this case k should equal 3

    truthvalues.shape = (m,) 
    predictions.shape = (m, k)
    '''
    z = (predictions == truthvalues[:, None]).astype(np.float32)
    weights = 1./(np.arange(predictions.shape[1], dtype=np.float32) + 1.)
    z = z * weights[None, :]
    return np.mean(np.sum(z, axis=1))

In [9]:
validate_in_bin = None
test_in_bin = None
train_in_bin_2 = None
x_train = None
x_validate = None
x_test = None
predicted_validate_place_id = None

In [10]:
def CV_fit(train, test, kfold=5, keep_fraction=0.5):
    '''Performs a fit using XGBoost. Applies kfold cross validation to 
    estimate error. 

    Parameters
    ----------
    train 
        The training dataset as a pandas DataFrame
    test
        The testing dataset as a pandas DataFrame
    kfold
        The number of folds to use for K Fold Validation
    keep_fraction
        A float between 0 and 1. The fraction of events in each bin
        to keep while minimizing the number of place_ids in the training
        set. Low values throw away a lot of infrequent place_ids, and
        values near 1 retain almost all place_ids. 
    '''
    global validate_in_bin
    global test_in_bin
    global train_in_bin_2
    global x_train
    global x_validate
    global x_test
    global predicted_validate_place_id

    # choose 10 random bins for a quicker estimate of algorithm error.
    run_all = False
    bin_numbers = []
    n_cell_y = 50
    n_cell_x = 50
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    
    if run_all:
        for i in range(0, n_cell_x):
            for j in range(0, n_cell_y):
                bin_numbers.append((i, j))
    else:
        NBINS = 1
        rs = np.random.RandomState(45)
        bin_numbers = zip(rs.randint(0, n_cell_x, size=NBINS), rs.randint(0, n_cell_x, size=NBINS))
    
    predictions = []
    map3s = []

    #Choose this line for the whole dataset.
    #for i_bin_x, i_bin_y in itertools.product(xrange(50), xrange(50)):
    for i_bin_x, i_bin_y in bin_numbers:
        print("Bin {},{}".format(i_bin_x, i_bin_y))

        # choose the correct bin, sort values in time to better simulate
        # the real train/test split for k-fold validation
        #train_in_bin = train[(train.x_bin_050 == i_bin_x)
        #                     & (train.y_bin_050 == i_bin_y)].sort_values('time')
        #test_in_bin = test[(test.x_bin_050 == i_bin_x)
        #                   & (test.y_bin_050 == i_bin_y)].sort_values('time')
                                   
                                   
        min_x = i_bin_x * x_length
        max_x = (i_bin_x + 1) * x_length
        min_y = i_bin_y * y_length
        max_y = (i_bin_y + 1) * y_length
    
        # include the edge
        if(i_bin_y + 1 == n_cell_y):
            max_y += 0.1
        if(i_bin_x + 1 == n_cell_x):
            max_x += 0.1
        train_in_bin = train[(train.time <= 786239 * 0.875) & \
                             (train.x >= min_x - 0.1) & \
                             (train.x < max_x + 0.1) & \
                             (train.y >= min_y - 0.1) & \
                             (train.y < max_y + 0.1)].copy()
                                   
        validate_in_bin = train[(train.time > 786239 * 0.875) & \
                                (train.x >= min_x) & \
                                (train.x < max_x) & \
                                (train.y >= min_y) & \
                                (train.y < max_y)].copy()
        
        test_in_bin = test[(test.x >= min_x) & \
                           (test.x < max_x) & \
                           (test.y >= min_y) & \
                           (test.y < max_y)].copy()

        N_total_in_bin = train_in_bin.shape[0]
        keep_N = int(float(N_total_in_bin)*keep_fraction)
        vc = train_in_bin.place_id.value_counts()

        # eliminate all ids which are low enough frequency
        vc = vc[np.cumsum(vc.values) < keep_N]
        df1 = pd.DataFrame({'place_id': vc.index, 'freq': vc.values})

        # this represents the training set after all low frequency place_ids
        # are removed
        train_in_bin_2 = pd.merge(train_in_bin, df1, on='place_id',
                                  how='inner')

        # XG Boost requires labels from 0 to n_labels, not place_ids
        le = preprocessing.LabelEncoder()
        le.fit(train_in_bin_2.place_id.values)
        y_train = le.transform(train_in_bin_2.place_id.values)

        # select columns (features) and make a numpy array
        x_train = train_in_bin_2[feature_list].as_matrix()
        x_validate = validate_in_bin[feature_list].as_matrix()
        x_test = test_in_bin[feature_list].as_matrix()

        # Construct DMatrices
        dm_train = xgboost.DMatrix(x_train, label=y_train)
        dm_validate = xgboost.DMatrix(x_validate)
        dm_test = xgboost.DMatrix(x_test)

        # use the XGBoost built in cross validation function,
        # stopping early to prevent overfitting
        #res = xgboost.cv(
        #    {'eta': 0.1, 'objective': 'multi:softprob',
        #     'num_class': len(le.classes_),
        #     'alpha': 0.1, 'lambda': 0.1, 'booster': 'gbtree'},
        #    dm_train, num_boost_round=200, nfold=kfold, seed=42,
        #    early_stopping_rounds=10, verbose_eval=10
        #    # For some reason, verbose_eval seems to be broken on my install
        #)
#
        #print(res)

        # this will be the number of epochs that (approximately) prevents
        # overfitting
        # N_epochs = res.shape[0]

        # For some reason, verbose_eval seems to be broken on my install
        booster = xgboost.train(
            {'eta': 0.1, 'objective': 'multi:softprob',
             'num_class': len(le.classes_),
             'alpha': 0.1, 'lambda': 0.1, 'booster': 'gbtree'},
            dm_train, num_boost_round=50, verbose_eval=10)
        predict_y_validate = booster.predict(dm_validate)
        predict_y_test = booster.predict(dm_test)

        # A top k algorithm would be theoretically faster, but benchmarks
        # indicate that with Order(50) elements, an argsort is faster than
        # heapq.

        predicted_validate_idx = np.argsort(
            predict_y_validate, axis=1)[:, -3:][:, ::-1]
        predicted_test_idx = np.argsort(
            predict_y_test, axis=1)[:, -3:][:, ::-1]

        c = np.array(le.classes_)
        predicted_validate_place_id = np.take(c, predicted_validate_idx)
        predicted_test_place_id = np.take(c, predicted_test_idx)

        map3 = mapkprecision(validate_in_bin.place_id, predicted_validate_place_id)
        map3s.append(map3)

        print("Train Cross Validated MAP@3: {0:.4f}".format(map3))
        result = pd.DataFrame({'row_id': test_in_bin.index,
                               'pred_1': predicted_test_place_id[:, 0],
                               'pred_2': predicted_test_place_id[:, 1],
                               'pred_3': predicted_test_place_id[:, 2]})
        predictions.append(result)

    return pd.concat(predictions), map3s


def main():

    print('Reading csv files from disk.', flush=True)
    train = pd.read_csv('../../train.csv')
    test = pd.read_csv('../../test.csv')

    print("Calculating time features", flush=True)

    #train['hour'] = (train['time']//60) % 24+1  # 1 to 24
    #test['hour'] = (test['time']//60) % 24+1  # 1 to 24
    
    train = prepare_data(train)
    test = prepare_data(test)

    print("Starting CV", flush=True)
    predictions, map3s = CV_fit(train, test, kfold=5)
    print("MAP@3 CV in bins {}".format(map3s), flush=True)

    print("Writing xgb_submission.csv", flush=True)
    with open('xgb_submission.csv', 'w') as fh:
        fh.write('row_id,place_id\n')
        for r in predictions.itertuples():
            fh.write("{0},{1} {2} {3}\n".format(*r))

if __name__ == '__main__':
    start_time = time.time()
    main()
    print("Elapsed time overall: %s seconds" % (time.time() - start_time), flush = True)

Reading csv files from disk.
Calculating time features
Starting CV
Bin 11,30
Train Cross Validated MAP@3: 0.4115
MAP@3 CV in bins [0.41146457]
Writing xgb_submission.csv
Elapsed time overall: 213.45775485038757 seconds


In [ ]:
    global validate_in_bin
    global test_in_bin
    global train_in_bin_2
    global x_train
    global x_validate
    global x_test
    global predicted_validate_place_id

In [15]:
predicted_validate_place_id

array([[7342780196, 6946999901, 2925870119],
       [6669101872, 2054638269, 5477482178],
       [1040521354, 8872038383, 6946999901],
       ..., 
       [7276935306, 4039268449, 2334694217],
       [7852420223, 1436469376, 9631172353],
       [6946999901, 4063850020, 2925870119]], dtype=int64)

In [17]:
predicted_df = pd.DataFrame(predicted_validate_place_id)

In [19]:
predicted_df.head()

,0,1,2
0,7342780196,6946999901,2925870119
1,6669101872,2054638269,5477482178
2,1040521354,8872038383,6946999901
3,5477482178,4211808004,6133381876
4,8153977514,6133381876,5477482178


In [16]:
validate_in_bin

,row_id,x,y,accuracy,time,place_id,hour,weekday,day,month,year,log_month
8699,8699,2.2790,6.0621,2.541579,690590,4063850020,22.850000,6,116,4,2,1.278431
26142,26142,2.3264,6.1431,1.799341,699436,9098017187,2.283333,6,123,5,2,1.283090
73911,73911,2.3812,6.0265,0.301030,700421,3828113927,18.700000,6,123,5,2,1.283605
99887,99887,2.2133,6.1764,1.301030,710622,6133381876,20.716667,6,130,5,2,1.288910
102264,102264,2.2611,6.1796,2.173186,705273,8153977514,3.566667,3,127,5,2,1.286137
103364,103364,2.3669,6.1956,1.806180,747830,2563739387,16.850000,4,156,6,2,1.307729
105442,105442,2.2360,6.0296,1.826075,731520,4577744390,9.016667,0,145,5,2,1.299580
120359,120359,2.3024,6.1835,2.060698,776347,1436469376,12.133333,3,176,6,2,1.321619
126352,126352,2.2110,6.1429,1.851258,689457,7741743260,3.966667,6,116,4,2,1.277830
135274,135274,2.2579,6.1874,1.799341,758933,1436469376,9.900000,5,164,6,2,1.313190
